In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
tr_mcc_codes = pd.read_csv('/content/drive/MyDrive/data/tr_mcc_codes.csv', sep=';')
tr_types = pd.read_csv('/content/drive/MyDrive/data/tr_types.csv', sep=';')
transactions = pd.read_csv('/content/drive/MyDrive/data/transactions.csv', nrows = 1000000)
gender_train = pd.read_csv('/content/drive/MyDrive/data/gender_train.csv')

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM' *(возможно и русское написание для ATM)*

Не использовать **merge**

In [5]:
random_transactions = transactions.sample(n = 1000, random_state=123)

contains_conditions = tr_types['tr_description'].str.contains('POS') | tr_types['tr_description'].str.contains('ATM') | tr_types['tr_description'].str.contains('АТМ')
frame_conditions = tr_types[contains_conditions]

search_in_random_transactions = random_transactions['tr_type'].isin(frame_conditions['tr_type'])
merged_frames = random_transactions[search_in_random_transactions]

print(len(merged_frames)/len(random_transactions))


0.633


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [6]:
frequency = pd.DataFrame(transactions.groupby(['tr_type']).size())
all_transaction_types = len(transactions)

frequency /= all_transaction_types
frequency *= 100

frequency.columns = ['frequency']
frequency.nlargest(10, 'frequency')

,frequency
tr_type,
1010,23.1117
2010,15.1166
7070,14.9006
1110,13.7658
1030,11.8975
2370,4.9830
7010,2.8972
7030,2.6078
7071,1.5781


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту (для клиента - положительный amount - затем суммирование)
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте (аналогично с приходом)
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов (для каждого из найденных в 1. и 2.) (гипотетически, в 1 и 2 может быть один человек)

In [7]:
max_income = pd.DataFrame(transactions[transactions['amount'] > 0].groupby(['customer_id'])['amount'].sum())
max_income.nlargest(1, 'amount')

,amount
customer_id,
70780820,1.248115e+09


In [8]:
max_outcome = pd.DataFrame(transactions[transactions['amount'] < 0].groupby(['customer_id'])['amount'].sum())
max_outcome.nsmallest(1, 'amount')

,amount
customer_id,
70780820,-1.249952e+09


In [9]:
max_income_index = max_income.nlargest(1, 'amount').index.values[0]
max_outcome_index = max_outcome.nsmallest(1, 'amount').index.values[0]

first_income = max_income.loc[max_income_index]['amount']
first_outcome = max_outcome.loc[max_income_index]['amount']

first_income + first_outcome


-1837317.979973793

In [10]:
second_income = max_income.loc[max_outcome_index]['amount']
second_outcome = max_outcome.loc[max_outcome_index]['amount']

second_income + second_outcome

-1837317.979973793

## Дополнительное задание
* Найдите пол клиента из задания 3.

In [11]:
get_max_income_gender = gender_train[gender_train['customer_id'] == max_income_index]['gender'].to_list()[0]

get_max_income_gender

1

* Найдите категории (mcc), по которым этот клиент совершал траты.

In [43]:
get_mcc = transactions[(transactions['customer_id'] == max_income_index) & 
                       (transactions['amount'] < 0)][['mcc_code']]

list(get_mcc['mcc_code'].unique())

[4829, 6011, 6010]

* Посчитайте средние траты по этим категориям у представителей этого пола.

In [69]:
get_specific_gender = gender_train[gender_train['gender'] == get_max_income_gender]
mean_mcc = pd.DataFrame(transactions[(transactions['customer_id'].isin(get_specific_gender['customer_id'])) 
             & (transactions['mcc_code'].isin(get_mcc['mcc_code']))
             & (transactions['amount'] < 0)]
             .groupby(['mcc_code'])['amount'].mean())
mean_mcc.columns = ['gender_mean']

mean_mcc

,gender_mean
mcc_code,
4829,-293647.370976
6010,-947212.341179
6011,-172598.191419


* В каких категориях у этого клиента нетипичные (в смысле отличия от средних трат по полу в обе стороны) траты?

In [80]:
client_transactions = transactions[(transactions['customer_id'] == max_income_index)
                     & (transactions['amount'] < 0)]
client_mean = pd.DataFrame(client_transactions.groupby('mcc_code')['amount'].mean())
client_mean.columns = ['client_mean']

with_mean = pd.merge(client_mean, mean_mcc, on='mcc_code', how='inner')
with_mean['diff'] = abs(abs(with_mean['client_mean']) - abs(with_mean['gender_mean']))

with_mean.sort_values(by='diff', ascending=False).index.to_list()

[6011, 4829, 6010]

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [50]:
task_2 = pd.DataFrame(frequency.nlargest(10, 'frequency').index.values)
task_2.columns = ['tr_type']

average_2 = pd.DataFrame(transactions[transactions['tr_type'].isin(task_2['tr_type'])].groupby('tr_type')['amount'].mean())
average_2.columns = ['average_amount']
average_2


,average_amount
tr_type,
1010,-19784.748640
1030,-5320.980222
1100,-44061.827262
1110,-32119.330371
2010,-136077.629325
2370,-205418.249032
7010,276391.789596
7030,86104.332909
7070,65569.831700


In [51]:
median_2 = pd.DataFrame(transactions[transactions['tr_type'].isin(task_2['tr_type'])].groupby('tr_type')['amount'].median())
median_2.columns = ['median_amount']
median_2

,median_amount
tr_type,
1010,-7411.52
1030,-2245.92
1100,-10188.26
1110,-11207.57
2010,-44918.32
2370,-44918.32
7010,112295.79
7030,13951.52
7070,11319.42


In [52]:
average_3_income = pd.DataFrame(transactions[transactions['customer_id'] == int(max_income_index)].groupby('tr_type')['amount'].mean())
average_3_income.columns = ['average_amount']
average_3_income

,average_amount
tr_type,
2010,-2.941056e+06
2330,-2.382398e+06
2370,-2.218505e+06
6110,1.756293e+04
7020,1.465129e+04
7021,2.850067e+04
7030,1.406196e+04
7031,1.921036e+04
7034,1.430648e+04


In [53]:
median_3_income = pd.DataFrame(transactions[transactions['customer_id'] == int(max_income_index)].groupby('tr_type')['amount'].median())
median_3_income.columns = ['median_amount']
median_3_income

,median_amount
tr_type,
2010,-3368873.660
2330,-2245915.770
2370,-2245915.770
6110,10028.010
7020,10319.985
7021,28500.670
7030,8848.910
7031,12992.620
7034,14306.480


In [54]:
average_3_outcome = pd.DataFrame(transactions[transactions['customer_id'] == int(max_outcome_index)].groupby('tr_type')['amount'].mean())
average_3_outcome.columns = ['average_amount']
average_3_outcome

,average_amount
tr_type,
2010,-2.941056e+06
2330,-2.382398e+06
2370,-2.218505e+06
6110,1.756293e+04
7020,1.465129e+04
7021,2.850067e+04
7030,1.406196e+04
7031,1.921036e+04
7034,1.430648e+04


In [55]:
median_3_outcome = pd.DataFrame(transactions[transactions['customer_id'] == int(max_outcome_index)].groupby('tr_type')['amount'].median())
median_3_outcome.columns = ['median_amount']
median_3_outcome

,median_amount
tr_type,
2010,-3368873.660
2330,-2245915.770
2370,-2245915.770
6110,10028.010
7020,10319.985
7021,28500.670
7030,8848.910
7031,12992.620
7034,14306.480


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [23]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (ноликами и единичками) (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [24]:
difference_between_outcome = pd.DataFrame(transactions[transactions['amount'] < 0].groupby(['gender'])['amount'].mean())
gender_list = difference_between_outcome.index.to_list()
abs(abs(difference_between_outcome.loc[gender_list[0]]['amount']) - abs(difference_between_outcome.loc[gender_list[1]]['amount']))

32718.05492021877

In [25]:
difference_between_income = pd.DataFrame(transactions[transactions['amount'] > 0].groupby(['gender'])['amount'].mean())
abs(abs(difference_between_income.loc[gender_list[0]]['amount']) - abs(difference_between_income.loc[gender_list[1]]['amount']))

63366.57104801515

## Задание 6

1. По всем типам транзакций tr_type рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

In [26]:
max_income = pd.DataFrame(transactions[transactions['amount'] > 0].groupby(['tr_type', 'gender'])['amount'].max()).sort_values(by='amount', ascending=False).reset_index()
gender_0 = max_income[max_income['gender'] == gender_list[0]].tail(10).sort_values(by='amount')

gender_0

,tr_type,gender,amount
71,4051,0.0,1122.96
69,4210,0.0,2245.92
70,4110,0.0,2245.92
66,2370,0.0,2335.75
65,4100,0.0,5041.86
64,7075,0.0,6737.75
63,2110,0.0,22459.16
62,1010,0.0,24839.83
60,2010,0.0,44918.32
58,2210,0.0,68315.82


In [27]:
gender_1 = max_income[max_income['gender'] == gender_list[1]].tail(10).sort_values(by='amount')

gender_1

,tr_type,gender,amount
67,2020,1.0,2245.92
68,4110,1.0,2245.92
61,1010,1.0,28803.87
59,8100,1.0,64682.37
57,6000,1.0,79280.83
55,7041,1.0,87590.72
54,2010,1.0,89836.63
53,2110,1.0,112295.79
51,2370,1.0,113194.15
50,7015,1.0,121279.45


In [28]:
equals_tr_types = gender_0[gender_0['tr_type'].isin(gender_1['tr_type'])]['tr_type'].to_list()

equals_tr_types

[4110, 2370, 2110, 1010, 2010]

## Задание 7

1. Найдите суммы затрат по каждой категории mcc для мужчин и для женщин. 
2. Найдите топ 5 категорий mcc с самыми большими относительными модулями разности в тратах для разных полов.

In [29]:
mcc_outcome = pd.DataFrame(transactions[transactions['amount'] < 0].groupby(['gender', 'mcc_code'])['amount'].sum()).reset_index()
mcc_outcome_0 = mcc_outcome[mcc_outcome['gender'] == gender_list[0]]

mcc_outcome_0

,gender,mcc_code,amount
0,0.0,742,-386207.68
1,0.0,1711,-674897.69
2,0.0,1799,-261626.73
3,0.0,2741,-20746.42
4,0.0,3000,-74431645.27
...,...,...,...
169,0.0,8699,-521107.48
170,0.0,8999,-19432057.92
171,0.0,9222,-361143.24
172,0.0,9311,-406197.45


In [30]:
mcc_outcome_1 = mcc_outcome[mcc_outcome['gender'] == gender_list[1]]

mcc_outcome_1

,gender,mcc_code,amount
174,1.0,742,-132277.93
175,1.0,1711,-206264.90
176,1.0,1731,-121542.67
177,1.0,1799,-1129257.68
178,1.0,2741,-221290.08
...,...,...,...
344,1.0,8999,-28643346.63
345,1.0,9222,-988205.17
346,1.0,9311,-373271.22
347,1.0,9399,-1810820.67


In [31]:
mcc_diff = pd.merge(mcc_outcome_0, mcc_outcome_1, on='mcc_code', suffixes=('_0', '_1'))
mcc_diff['diff'] = abs(mcc_diff['amount_0'] - mcc_diff['amount_1'])

mcc_codes_top_5 = mcc_diff.sort_values(by='diff').tail(5)
mcc_codes_top_5['mcc_code'].to_list()

[5511, 5541, 6010, 4829, 6011]

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount > 0 в ночное время для мужчин и женщин. Ночное - это примерно 00-06 часов. 

In [32]:
transactions['tr_hour'] = transactions['tr_datetime'].str.extract(r'\d (\d{2}):\d{2}:\d{2}')

transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,07
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,07
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,08
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14
...,...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,23
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,19
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,22
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,23


In [33]:
transactions[(transactions['amount'] > 0) & 
             (pd.to_numeric(transactions['tr_hour']) <= 6) &  
             (pd.to_numeric(transactions['tr_hour']) >= 0)].groupby('gender')['tr_type'].count()

gender
0.0     4059
1.0    32726
Name: tr_type, dtype: int64